NOTA IMPORTANTE: Si el signo "!" no sirve para ejecutar comando de consola se debe actualizar la siguiente libreria
pip install --upgrade click

In [ ]:
#!pip install spacy
#!pip install spacy-transformers
#!pip install spacy[cuda100]

In [ ]:
#!python -m spacy download es_core_news_sm
!python -m spacy info

In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("es")
db = DocBin()

In [4]:
#spacy.prefer_gpu()
#spacy.require_gpu()

False

Página para taggear palabras
Aqui va el archivo json ya "segmentado"
pagina para segmentar de spacy 
https://tecoholic.github.io/ner-annotator/

In [ ]:
import json 
tdata = 'datos_training.json'
vdata = 'datos_validation.json'


f = open(vdata,"r",encoding="utf-8")
TRAIN_DATA = json.load

In [23]:

print(TRAIN_DATA["annotations"][0])

['"Pofi, reproduce",Adiós a Jamaica (Jamaica Farewell),del,Los Hooligans\r', {'entities': [[1, 5, 'POFI'], [7, 16, 'ORDEN'], [18, 33, 'CANCION'], [53, 56, 'PREPOSICION'], [57, 70, 'ARTISTA']]}]


In [ ]:
TRAIN_DATA

In [ ]:
for text, annot in tqdm(TRAIN_DATA["annotations"]):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
          print("salto de entidad")
        else:
          ents.append(span)
    doc.ents = ents
    db.add(doc)

#el json hay que convertirlo a .spacy porque es la unica forma en que esa librería
#lo pueda leer 
db.to_disk("datos_validation.spacy")

In [ ]:
from spacy.util import filter_spans

for text,training_example  in tqdm(TRAIN_DATA['annotations']): 
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    db.add(doc)

db.to_disk("validation_data.spacy") # save the docbin object

Este segmento es para crear el archivo config
este archivo config también se puede descargar de
https://spacy.io/usage/training

El archivo config que está debajo es creado mediante código y usa la CPU para entrenar el modelo por lo tanto no tiene la arquitectura transformer

In [25]:

!python -m spacy init config config.cfg --lang es --pipeline ner --optimize efficiency

ℹ Generated config template specific for your use case
- Language: es
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


c:\Users\Oscar\AppData\Local\Programs\Python\Python39\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\Users\Oscar\AppData\Local\Programs\Python\Python39\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\Users\Oscar\AppData\Local\Programs\Python\Python39\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\Users\Oscar\AppData\Local\Programs\Python\Python39\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


Este segundo archivo config es descargado de la página y configurado con el idioma español, NER, GPU para que use la arquitectura transformer

In [5]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


El primer archivo es para entrenar, el segundo para validar

In [3]:

!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./validation_data.spacy

[i] Saving to output directory: .
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     63.95    0.00    0.00    0.00    0.00
  2     200        147.99   1488.29   98.62   98.28   98.95    0.99
  5     400        235.47    228.68   99.09   99.07   99.11    0.99
  8     600        207.94    194.89   99.01   98.89   99.13    0.99
 13     800        324.61    179.56   98.95   98.97   98.93    0.99
 18    1000        398.58    209.67   99.25   99.29   99.20    0.99
 24    1200        419.32    190.10   99.16   99.16   99.16    0.99
 32    1400        301.17    141.83   99.32   99.36   99.27    0.99
 42    1600        203.08 

C:\Users\Oscar\AppData\Local\Programs\Python\Python39\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\Oscar\AppData\Local\Programs\Python\Python39\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\Oscar\AppData\Local\Programs\Python\Python39\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\Oscar\AppData\Local\Programs\Python\Python39\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
[2023-05-14 22:24:31,296] [INFO] Set up nlp object from config
[2023-05-14 22:24:31,308] [INFO] Pipeline: ['tok2vec', 'ner']

In [4]:
nlp_ner = spacy.load("model-last")

In [143]:
doc = nlp_ner('Pofi, tocame", "lejo de ti",de, kaden')

In [144]:
spacy.displacy.render(doc, style="ent", jupyter=True)